In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, Conv2D, Flatten, Concatenate
from tensorflow.keras.models import Model
from transformers import TFBertModel, BertTokenizer
import numpy as np
import pandas as pd
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import re

2023-12-05 17:36:28.257526: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


/Users/xinyutian/opt/anaconda3/envs/tf/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/xinyutian/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
raw_data = pd.read_csv('../raw_data/train.csv')

In [3]:
prices = raw_data['price']

In [4]:
text_cols = ['name', 
             'host_name',
            'description', 
            'host_verifications', 
            'neighbourhood_cleansed', 
            'neighbourhood_group_cleansed',
            'property_type',
            'room_type',
            'bathrooms_text',
            'amenities']
text_data = raw_data[text_cols]
text_data.head()

,name,host_name,description,host_verifications,neighbourhood_cleansed,neighbourhood_group_cleansed,property_type,room_type,bathrooms_text,amenities
0,Home in Torrance · 1 bedroom · 1 bed · 1 share...,Leonardo,"Beautifully designed decor, lighting and enter...","['email', 'phone']",Harbor Gateway,City of Los Angeles,Shared room in home,Shared room,1 shared bath,"[""Stove"", ""Iron"", ""Window guards"", ""Smoking al..."
1,Rental unit in Venice · ★4.74 · 2 bedrooms · 2...,Donald,The location is great - the apt is HUGH 1877 S...,"['email', 'phone']",Venice,City of Los Angeles,Entire rental unit,Entire home/apt,2 baths,"[""Stove"", ""Carbon monoxide alarm"", ""Iron"", ""Fr..."
2,Home in Avalon · ★5.0 · 3 bedrooms · 4 beds · ...,Catalina Vacations,"315 Eucalyptus Lower<br />Spacious, Remodeled ...","['email', 'phone', 'work_email']",Avalon,Other Cities,Entire home,Entire home/apt,2 baths,"[""Stove"", ""Carbon monoxide alarm"", ""Iron"", ""Ho..."
3,Home in Santa Clarita · 4 bedrooms · 5 beds · ...,Kellie,Close to almost everything when you stay at th...,"['email', 'phone']",Santa Clarita,Other Cities,Entire home,Entire home/apt,2.5 baths,"[""Free parking on premises"", ""Smoke alarm"", ""C..."
4,Hotel in Los Angeles · ★New · 1 bedroom · 1 be...,RoomPicks,Enjoy the thrill of staying in a hotel located...,"['email', 'phone']",Westwood,City of Los Angeles,Room in hotel,Private room,1 bath,"[""Carbon monoxide alarm"", ""Iron"", ""Air conditi..."


In [5]:
def preprocess_text(text, placeholder_token="[NO_DESCRIPTION]"):
    if type(text) != str:
        # Replace missing values with empty strings
        text = ""
    if not text or text.isspace():
        # Replace empty/whitespace-only text with placeholder
        return placeholder_token
    else:
        text = text.lower()
        text = re.sub(r"[^\w\s]", "", text)
        text = re.sub(r"\s+", " ", text).strip()
        text = re.sub(r"[^a-zA-Z]+", " ", text)
        text = re.sub(r'[^\w\s]', '', text)
        text = re.sub(r"\s+", " ", text).strip()
        text = re.sub(r"^\s+$", "", text)
        text = re.sub(r"^\s+", "", text)
        text = re.sub(r"\s+$", "", text)
        # remove stop words
        stop_words = stopwords.words('english')
        text = text.split()
        text = [word for word in text if not word in stop_words]
        text = ' '.join(text)
    return text

for col in text_data.columns:
    text_data[col] = text_data[col].apply(lambda x: preprocess_text(x))

/var/folders/46/46qfn8750nz4wpqzldyf5qf40000gn/T/ipykernel_39721/277857435.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text_data[col] = text_data[col].apply(lambda x: preprocess_text(x))


In [6]:
text_data.head()

,name,host_name,description,host_verifications,neighbourhood_cleansed,neighbourhood_group_cleansed,property_type,room_type,bathrooms_text,amenities
0,home torrance bedroom bed shared bath,leonardo,beautifully designed decor lighting entertainm...,email phone,harbor gateway,city los angeles,shared room home,shared room,shared bath,stove iron window guards smoking allowed micro...
1,rental unit venice bedrooms beds baths,donald,location great apt hugh sf area cal king beds ...,email phone,venice,city los angeles,entire rental unit,entire homeapt,baths,stove carbon monoxide alarm iron freezer dinin...
2,home avalon bedrooms beds baths,catalina vacations,eucalyptus lowerbr spacious remodeled avalon h...,email phone work email,avalon,cities,entire home,entire homeapt,baths,stove carbon monoxide alarm iron host greets m...
3,home santa clarita bedrooms beds baths,kellie,close almost everything stay centrallylocated ...,email phone,santa clarita,cities,entire home,entire homeapt,baths,free parking premises smoke alarm carbon monox...
4,hotel los angeles new bedroom bed bath,roompicks,enjoy thrill staying hotel located famed unive...,email phone,westwood,city los angeles,room hotel,private room,bath,carbon monoxide alarm iron air conditioning lu...


In [7]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokenized_data = [tokenizer.encode_plus(text, 
                                        add_special_tokens=True,
                                        max_length=128,
                                        return_attention_mask=True,
                                        return_tensors='tf')
                  for column in text_data.columns for text in text_data[column]]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [8]:
len(tokenized_data)

154410

In [9]:
tokenized_data[0]

{'input_ids': <tf.Tensor: shape=(1, 9), dtype=int32, numpy=
array([[  101,  2188, 17153, 21621,  5010,  2793,  4207,  7198,   102]],
      dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 9), dtype=int32, numpy=array([[0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 9), dtype=int32, numpy=array([[1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=int32)>}

In [10]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [11]:
input_ids = [pad_sequences([column_data['input_ids']], 
                           maxlen=128, 
                           truncating='post', 
                           padding='post') for column_data in tokenized_data]

In [12]:
attention_masks = [pad_sequences([column_data['attention_mask']], 
                                 maxlen=128, 
                                 truncating='post', 
                                 padding='post') for column_data in tokenized_data]

In [40]:
# Define model parameters
max_length = 128  # or some other value depending on your text data

def build_bert_model(max_length):
    # Prepare inputs for each text column
    bert_inputs = []
    for i in range(len(text_data.columns)):  # For 7 text columns
        input_ids = Input(shape=(max_length,), dtype='int32', name=f'input_ids_{i}')
        attention_masks = Input(shape=(max_length,), dtype='int32', name=f'attention_mask_{i}')
        bert_inputs.extend([input_ids, attention_masks])

    # Load BERT model
    bert_model = TFBertModel.from_pretrained('bert-base-uncased')

    # BERT processing for each column
    bert_outputs = []
    for i in range(0, len(bert_inputs), 2):
        input_ids, attention_masks = bert_inputs[i], bert_inputs[i + 1]
        bert_output = bert_model(input_ids, attention_mask=attention_masks)[1]
        bert_outputs.append(bert_output)

    # Concatenate all BERT outputs
    concatenated_outputs = Concatenate()(bert_outputs)
    bert_dense = Dense(64, activation='relu')(concatenated_outputs)

    return bert_inputs, bert_dense

In [14]:
image_data = np.load('../train_images.npy')
ids = np.load('../train_ids.npy')

In [15]:
image_data = image_data / 255

In [16]:
from tensorflow.keras.regularizers import l2
# CNN model for image data
def build_cnn_model():
    image_input = Input(shape=(224, 224, 3))
    cnn_layer = Conv2D(64, (3, 3), activation='relu')(image_input)
    cnn_flatten = Flatten()(cnn_layer)
    cnn_dense = Dense(64, activation='relu', kernel_regularizer=l2(0.01))(cnn_flatten)
    return image_input, cnn_dense

In [17]:
num_data = pd.read_csv('../data_preprocess/train_data.csv')

In [18]:
num_data.drop(['host_since'], axis=1, inplace=True)

In [19]:
num_data.head()

,host_id,host_identity_verified,accommodates,price,minimum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_nights_avg_ntm,availability_30,availability_60,availability_90,availability_365,number_of_reviews_ltm,number_of_reviews_l30d
0,205539927,1,3,0.0,30,30,30,30.0,28,58,88,363,0,0
1,7599550,1,6,3.0,3,3,3,3.0,13,22,40,296,24,2
2,271118401,1,8,2.0,3,3,3,3.0,19,37,58,201,2,0
3,429951162,1,7,3.0,28,28,28,28.0,30,60,90,364,0,0
4,501999278,1,2,4.0,1,1,1,1.0,12,40,70,269,0,0


In [20]:
len(num_data.columns)

14

In [21]:
# LSTM model for numerical data
def build_lstm_model():
    numerical_input = Input(shape=(13, 1))
    lstm_layer = LSTM(32)(numerical_input)
    lstm_dense = Dense(64, activation='relu')(lstm_layer)
    return numerical_input, lstm_dense

In [41]:
# Combine all models
def build_ensemble_model():
    text_inputs, bert_output = build_bert_model(max_length)
    image_input, cnn_output = build_cnn_model()
    numerical_input, lstm_output = build_lstm_model()

    combined = Concatenate()([bert_output, cnn_output, lstm_output])
    final_dense = Dense(128, activation='relu')(combined)
    output = Dense(6, activation='softmax')(final_dense)  # 6 classes for price

    model = Model(inputs=text_inputs + [image_input, numerical_input], outputs=output)
    return model

In [23]:
image_data.shape

(15096, 224, 224, 3)

In [24]:
images = np.zeros((15441, 224, 224, 3))
images[ids] = image_data

In [25]:
image_data = images

In [28]:
image_labels = prices

In [30]:
image_labels

0        0.0
1        3.0
2        2.0
3        3.0
4        4.0
        ... 
15436    2.0
15437    3.0
15438    0.0
15439    1.0
15440    1.0
Name: price, Length: 15441, dtype: float64

In [31]:
from tensorflow.keras.utils import to_categorical
image_labels= to_categorical(image_labels, num_classes = 6)

In [32]:
len(image_labels)

15441

In [33]:
num_data.head()

,host_id,host_identity_verified,accommodates,price,minimum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_nights_avg_ntm,availability_30,availability_60,availability_90,availability_365,number_of_reviews_ltm,number_of_reviews_l30d
0,205539927,1,3,0.0,30,30,30,30.0,28,58,88,363,0,0
1,7599550,1,6,3.0,3,3,3,3.0,13,22,40,296,24,2
2,271118401,1,8,2.0,3,3,3,3.0,19,37,58,201,2,0
3,429951162,1,7,3.0,28,28,28,28.0,30,60,90,364,0,0
4,501999278,1,2,4.0,1,1,1,1.0,12,40,70,269,0,0


In [34]:
num_data.drop(['price'], axis=1, inplace=True)

In [35]:
len(text_data.columns)

10

In [36]:
from sklearn.model_selection import train_test_split

# Splitting textual data
train_input_ids, test_input_ids, train_attention_masks, test_attention_masks = [], [], [], []

for i in range(10):
    ids_train, ids_test, masks_train, masks_test = train_test_split(input_ids[i*15441:(i*15441 + 15441)], attention_masks[i*15441:(i*15441 + 15441)], test_size=0.2, random_state=42)
    train_input_ids.append(ids_train)
    test_input_ids.append(ids_test)
    train_attention_masks.append(masks_train)
    test_attention_masks.append(masks_test)

# Splitting image data
image_train, image_test, labels_train, labels_test = train_test_split(image_data, image_labels, test_size=0.2, random_state=42)

# Splitting numerical data
numerical_train, numerical_test = train_test_split(num_data, test_size=0.2, random_state=42)

# Ensure that labels are also split
y_train, y_test = labels_train, labels_test


In [54]:
# Combine training data
train_data = [train_input_ids[i] for i in range(10)] + [train_attention_masks[i] for i in range(10)] + [image_train, numerical_train]
test_data = [test_input_ids[i] for i in range(10)] + [test_attention_masks[i] for i in range(10)] + [image_test, numerical_test]

In [60]:
train_data[:][0]

[array([[[  101,  6992, 12218, ...,  9705, 19692,   102],
         [    0,     0,     0, ...,     0,     0,     0],
         [    0,     0,     0, ...,     0,     0,     0],
         ...,
         [    0,     0,     0, ...,     0,     0,     0],
         [    0,     0,     0, ...,     0,     0,     0],
         [    0,     0,     0, ...,     0,     0,     0]]], dtype=int32),
 array([[[  101,  2237,  4580, ...,  9705, 19692,   102],
         [    0,     0,     0, ...,     0,     0,     0],
         [    0,     0,     0, ...,     0,     0,     0],
         ...,
         [    0,     0,     0, ...,     0,     0,     0],
         [    0,     0,     0, ...,     0,     0,     0],
         [    0,     0,     0, ...,     0,     0,     0]]], dtype=int32),
 array([[[  101,  6644, 28794, 18390,  9705,  7198,   102],
         [    0,     0,     0,     0,     0,     0,     0],
         [    0,     0,     0,     0,     0,     0,     0],
         [    0,     0,     0,     0,     0,     0,     0],
    

In [64]:
train = []
test = []
for i in range(len(y_train)):
    train.append(train_data[:][0])
for i in range(len(y_test)):
    test.append(test_data[:][0])


In [66]:
# Compile and train the model
ensemble_model = build_ensemble_model()
ensemble_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = ensemble_model.fit(train, y_train, validation_data=(test, y_test), epochs=10, batch_size=32)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

: 